## Use MVTec AD Dataset via API

In [ ]:
import os
from pathlib import Path

import numpy as np
from git.repo import Repo
from PIL import Image
from torchvision.transforms import ToPILImage

from anomalib.data.mvtec import MVTec, MVTecDataset
from anomalib.data.utils import InputNormalizationMethod, get_transforms

## Setting up the Working Directory
This cell is to ensure we change the directory to anomalib source code to have access to the datasets and config files. We assume that you already went through `001_getting_started.ipynb` and install the required packages.

In [ ]:
current_directory = Path.cwd()
if current_directory.name == "100_datamodules":
    # On the assumption that, the notebook is located in
    #   ~/anomalib/notebooks/100_datamodules/
    root_directory = current_directory.parent.parent
elif current_directory.name == "anomalib":
    # This means that the notebook is run from the main anomalib directory.
    root_directory = current_directory
else:
    # Otherwise, we'll need to clone the anomalib repo to the `current_directory`
    repo = Repo.clone_from(url="https://github.com/openvinotoolkit/anomalib.git", to_path=current_directory)
    root_directory = current_directory / "anomalib"

os.chdir(root_directory)
mvtec_dataset_root = root_directory / "datasets/MVTec"

### DataModule

Anomalib data modules are based on PyTorch Lightning (PL)'s `LightningDataModule` class. This class handles all the boilerplate code related to subset splitting, and creating the dataset and dataloader instances. A datamodule instance can be directly passed to a PL Trainer which is responsible for carrying out Anomalib's training/testing/inference pipelines. 

In the current example, we will show how an Anomalib data module can be created for the MVTec Dataset, and how we can obtain training and testing dataloaders from it.

To create a datamodule, we simply pass the path to the root folder of the dataset on the file system, together with some basic parameters related to pre-processing and image loading:

In [ ]:
mvtec_datamodule = MVTec(
    root=mvtec_dataset_root,
    category="bottle",
    image_size=256,
    train_batch_size=32,
    eval_batch_size=32,
    num_workers=8,
    task="segmentation",
    normalization=InputNormalizationMethod.NONE,  # don't apply normalization, as we want to visualize the images
)

For the illustrative purposes of the current example, we need to manually call the `prepare_data` and `setup` methods. Normally it is not necessary to call these methods explicitly, as the PL Trainer would call these automatically under the hood.

`prepare_data` checks if the dataset files can be found at the specified file system location. If not, it will download the dataset and place it in the folder.

`setup` applies the subset splitting and prepares the PyTorch dataset objects for each of the train/val/test subsets.

In [ ]:
mvtec_datamodule.prepare_data()
mvtec_datamodule.setup()

After the datamodule has been set up, we can use it to obtain the dataloaders of the different subsets.

In [ ]:
# Train images
i, data = next(enumerate(mvtec_datamodule.train_dataloader()))
print(data.keys(), data["image"].shape)

In [ ]:
# Test images
i, data = next(enumerate(mvtec_datamodule.test_dataloader()))
print(data.keys(), data["image"].shape, data["mask"].shape)

As can be seen above, creating the dataloaders are pretty straghtforward, which could be directly used for training/testing/inference. We could visualize samples from the dataloaders as well.

In [ ]:
img = ToPILImage()(data["image"][0].clone())
msk = ToPILImage()(data["mask"][0]).convert("RGB")

Image.fromarray(np.hstack((np.array(img), np.array(msk))))

### Torch Dataset

In some cases it might be desirable to create a standalone PyTorch dataset without a PL data module. For example, this could be useful for training a PyTorch model outside Anomalib, so without the use of a PL Trainer instance. In such cases, the PyTorch Dataset instance can be instantiated directly.

In [ ]:
MVTecDataset??

To create `MVTecDataset` we need to create the albumentations object that applies transforms to the input image.

In [ ]:
get_transforms??

In [ ]:
image_size = (256, 256)
transform = get_transforms(image_size=256, normalization=InputNormalizationMethod.NONE)

#### Classification Task

In [ ]:
# MVTec Classification Train Set
mvtec_dataset_classification_train = MVTecDataset(
    root=mvtec_dataset_root,
    category="bottle",
    transform=transform,
    split="train",
    task="classification",
)
mvtec_dataset_classification_train.setup()  # like the datamodule, the dataset needs to be set up before use
mvtec_dataset_classification_train.samples.head()

In [ ]:
sample = mvtec_dataset_classification_train[0]
print(sample.keys(), sample["image"].shape)

As can be seen above, when we choose `classification` task and `train` split, the dataset only returns `image`. This is mainly because training only requires normal images and no labels. Now let's try `test` split for the `classification` task

In [ ]:
# MVTec Classification Test Set
mvtec_dataset_classification_test = MVTecDataset(
    root=mvtec_dataset_root,
    category="bottle",
    transform=transform,
    split="test",
    task="classification",
)
mvtec_dataset_classification_test.setup()  # like the datamodule, the dataset needs to be set up before use
sample = mvtec_dataset_classification_test[0]
print(sample.keys(), sample["image"].shape, sample["image_path"], sample["label"])

#### Segmentation Task

It is also possible to configure the MVTec dataset for the segmentation task, where the dataset object returns image and ground-truth mask.

In [ ]:
# MVTec Segmentation Train Set
mvtec_dataset_segmentation_train = MVTecDataset(
    root=mvtec_dataset_root,
    category="bottle",
    transform=transform,
    split="train",
    task="segmentation",
)
mvtec_dataset_segmentation_train.setup()
mvtec_dataset_segmentation_train.samples.head()

In [ ]:
# MVTec Segmentation Test Set
mvtec_dataset_segmentation_test = MVTecDataset(
    root=mvtec_dataset_root,
    category="bottle",
    transform=transform,
    split="test",
    task="segmentation",
)
mvtec_dataset_segmentation_test.setup()
sample = mvtec_dataset_segmentation_test[20]
print(sample.keys(), sample["image"].shape, sample["mask"].shape)

Let's visualize the image and the mask...

In [ ]:
img = ToPILImage()(sample["image"].clone())
msk = ToPILImage()(sample["mask"]).convert("RGB")

Image.fromarray(np.hstack((np.array(img), np.array(msk))))